In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
from collections import Counter
import string
import re
from nltk.corpus import stopwords

In [75]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [76]:
ents_data = data[data['entities'] != ''].copy()

In [77]:
from pathlib import Path

path_train = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_train_encodings.jsonl")
path_testa = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testa_encodings.jsonl")
path_testb = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testb_encodings.jsonl")
raw_encodings_train = open(path_train, 'r').read()
raw_encodings_testa = open(path_testa, 'r').read()
raw_encodings_testb = open(path_testb, 'r').read()

In [78]:
import json

jsonl_parsed_train = [json.loads(x) for x in raw_encodings_train.splitlines()]
jsonl_parsed_testa = [json.loads(x) for x in raw_encodings_testa.splitlines()]
jsonl_parsed_testb = [json.loads(x) for x in raw_encodings_testb.splitlines()]
jsonl_parsed = jsonl_parsed_train + jsonl_parsed_testa + jsonl_parsed_testb

In [79]:
encodings = [x['encoding'] for x in jsonl_parsed]

In [80]:
ents_data['encodings'] = encodings
ents_data_filtered = ch.filter_data(ents_data, 3)

In [81]:
ents_data = ents_data_filtered[ents_data_filtered['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
encodings = ents_data['encodings'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

In [82]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0_3.txt", dtype=np.int32)
lev_cluster_dict = {}
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], encodings[i]))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], encodings[i])]

In [83]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering, DBSCAN
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.035, linkage="single")
# clusterizator = DBSCAN(min_samples=0, metric="cosine", n_jobs=-1, eps=0.035)
for x in tqdm(lev_cluster_dict):
    encodings = [y[2] for y in lev_cluster_dict[x]]
    if len(encodings) > 1:
        cluster = clusterizator.fit_predict(encodings)
        cluster_all.append(cluster)
    else:
        cluster_all.append(np.zeros(1))

100%|██████████| 2750/2750 [00:02<00:00, 1036.56it/s]


In [84]:
clustered_entities = [[row[1] for row in lev_cluster_dict[index]] for index in lev_cluster_dict.keys()]

In [85]:
clusters_splitted = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_splitted.append(cluster_unique)

In [86]:
clusters_splitted_dict = []
for el in clusters_splitted:
    if len(el) == 1:
        clusters_splitted_dict.append(el[0])
    else:
        max_len = max([len(el[key]) for key in el])

        indices = [i for i,d in enumerate([len(el[key]) for key in el]) if d==max_len]
        for i in indices:
            clusters_splitted_dict.append(el[i])

In [87]:
clusters_splitted_dict = [dict(Counter(x)) for x in clusters_splitted_dict]

### Gold standard

In [88]:
golden_standard_dict = ch.get_gold_standard_dict(ents_data)

### B-CUBED

In [89]:
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for cluster in clusters_splitted_dict:
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(cluster[gold_key], 2) /
                                                           sum(cluster.values()))
        except:
            pass
bcubed_precision = bcubed_precision_num / sum([sum(x.values()) for x in clusters_splitted_dict])
bcubed_precision

100%|██████████| 2089/2089 [00:01<00:00, 1504.00it/s]


0.8922369857472194

In [90]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for cluster in clusters_splitted_dict:
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(cluster[gold_key], 2) /
                                                     golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num / ents_data.shape[0]
bcubed_recall

100%|██████████| 2089/2089 [00:01<00:00, 1502.86it/s]


0.7808019609024331

In [91]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.8328083329514879

### evaluation levestein

CEAFm precision

In [92]:
max_evaluation_dict = dict.fromkeys(set(entities), 0)
for key in tqdm(max_evaluation_dict.keys()):
    for i in clusters_splitted_dict:
        try:
            if max_evaluation_dict[key] < i[key]:
                max_evaluation_dict[key] = i[key]
        except:
            pass

100%|██████████| 2089/2089 [00:01<00:00, 1511.50it/s]


In [94]:
# CEAFm_levenshtein_precision
CEAFm_levenshtein_precision = sum([x for x in max_evaluation_dict.values()]) / sum([sum(x.values()) for x in clusters_splitted_dict])
CEAFm_levenshtein_precision

0.8477996701675202

CEAFm recall

In [95]:
# CEAFm_levenshtein_recall
CEAFm_levenshtein_recall = sum([x for x in max_evaluation_dict.values()]) / ents_data.shape[0]
CEAFm_levenshtein_recall

0.8396011518459621

CEAFm Fscore

In [96]:
# CEAFm_levenshtein_f1
CEAFm_levenshtein_f1 = (2 * (CEAFm_levenshtein_recall * CEAFm_levenshtein_precision)) / (
        CEAFm_levenshtein_precision + CEAFm_levenshtein_recall)
CEAFm_levenshtein_f1

0.8436804940724265